In [ ]:
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

Набор данных должен быть вам знаком - это заявки в ЖКХ. Каждая строчка заявка, признаки - сколько раз встретилось то или иное слово в заявке, и есть результирующая переменная - это категория заявки.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/zayavki.csv')
df.head()

,httpgorodgovspbruproblems,аварийный,автомобиль,административный,администрация,адрес,арка,асфальт,асфальтовый,бак,...,элемент,этаж,это,являться,ям,яма,январь,ящик,id_category,category
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,10,Содержание МКД
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,Содержание МКД
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,Благоустройство
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,10,Содержание МКД
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23,Нарушение правил пользования общим имуществом


In [ ]:
#признаки
X=df.drop(['id_category','category'],axis=1)
scal_X = StandardScaler().fit_transform(X)
scal_X = pd.DataFrame(scal_X, columns=X.columns)
scal_X.shape

NameError: name 'df' is not defined

В случае результирующей переменной  в задаче многоклассовой классификации  для сетки важно, чтобы каждая уникальная метка была представлена отдельным столбцом

In [ ]:
encoder = LabelEncoder() #кодируем каждую метку класса числом
y = encoder.fit_transform(df['category'])
print(df['category'][:10].values)
print(y[:10])

['Содержание МКД' 'Содержание МКД' 'Благоустройство' 'Содержание МКД'
 'Нарушение правил пользования общим имуществом' 'Содержание МКД'
 'Благоустройство' 'Центральное отопление' 'Благоустройство'
 'Благоустройство']
[11 11  0 11  5 11  0 14  0  0]


In [ ]:
#как раскодить обратно
encoder.inverse_transform(y[:10])

array(['Содержание МКД', 'Содержание МКД', 'Благоустройство',
       'Содержание МКД', 'Нарушение правил пользования общим имуществом',
       'Содержание МКД', 'Благоустройство', 'Центральное отопление',
       'Благоустройство', 'Благоустройство'], dtype=object)

In [ ]:
y.shape

(59889,)

In [ ]:
#теперь y это массив с 15 столбцами, каждый столбец - уникальная метка класса
y = to_categorical(y) #превращаем в кучу столбцов
y.shape

(59889, 15)

In [ ]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
      dtype=float32)

In [ ]:
X_train_full,X_test,y_train_full,y_test = train_test_split(scal_X,y,test_size=0.2,random_state=0)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size=0.2,random_state=0)

In [ ]:
y.shape[1]

15

In [ ]:
model = keras.models.Sequential()
model.add(Dense(500, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
history=model.fit(X_train, y_train, epochs =5,batch_size=5)

Epoch 1/5
7666/7666 [==============================] - 58s 7ms/step - loss: 0.4645 - accuracy: 0.8716
Epoch 2/5
7666/7666 [==============================] - 55s 7ms/step - loss: 0.3281 - accuracy: 0.9031
Epoch 3/5
7666/7666 [==============================] - 52s 7ms/step - loss: 0.2897 - accuracy: 0.9159
Epoch 4/5
7666/7666 [==============================] - 53s 7ms/step - loss: 0.2596 - accuracy: 0.9246
Epoch 5/5
7666/7666 [==============================] - 53s 7ms/step - loss: 0.2482 - accuracy: 0.9301


In [ ]:
acc=model.evaluate(X_test,y_test)
print(np.array(acc)[1].round(2))

375/375 [==============================] - 1s 3ms/step - loss: 0.5386 - accuracy: 0.8901
0.89


In [ ]:
y_pred = model.predict(X_test)
y_pred[:10]

375/375 [==============================] - 1s 3ms/step


array([[9.99999940e-01, 0.00000000e+00, 2.84177593e-31, 1.55920968e-12,
        0.00000000e+00, 0.00000000e+00, 3.81990808e-35, 9.43436163e-29,
        8.59631377e-21, 0.00000000e+00, 0.00000000e+00, 2.21574422e-17,
        2.01450579e-31, 7.15549362e-37, 0.00000000e+00],
       [1.16240201e-07, 0.00000000e+00, 7.07150411e-15, 3.35320556e-11,
        1.45493284e-18, 9.99999464e-01, 4.06874364e-07, 4.55304324e-23,
        0.00000000e+00, 5.90456657e-22, 7.26363516e-20, 3.99398153e-10,
        1.18956653e-34, 1.17930415e-10, 2.16326881e-24],
       [8.65427613e-01, 3.94572534e-11, 4.92710805e-09, 6.32766387e-06,
        4.65930572e-10, 7.23533844e-07, 5.91931748e-04, 7.47458728e-07,
        2.24324049e-05, 2.62983146e-09, 2.64769522e-08, 2.62035523e-03,
        1.31251812e-01, 7.79716065e-05, 1.45970445e-16],
       [6.77049684e-05, 9.53878022e-15, 2.51008305e-06, 3.10326413e-07,
        7.31436739e-06, 6.55847325e-05, 9.45715538e-14, 0.00000000e+00,
        5.11226947e-15, 1.14379097e-0

In [ ]:
#метки класса
y_classes = y_pred.argmax(axis=-1)
y_classes[:10]

array([ 0,  5,  0, 11,  0,  0, 11,  0, 11,  0])

In [ ]:
#декодинг меток в название категорий
encoder.inverse_transform(y_classes[:10])

array(['Благоустройство', 'Нарушение правил пользования общим имуществом',
       'Благоустройство', 'Содержание МКД', 'Благоустройство',
       'Благоустройство', 'Содержание МКД', 'Благоустройство',
       'Содержание МКД', 'Благоустройство'], dtype=object)

## Лабораторная работа №2 - Keras

* тезисно - решаете задачу регрессии ИЛИ классификации
* Набор данных ищете сами, можете смело взять с прошлой лабы

Для классификации:
* использовать минимум две метрики для оценки качества классификатора
* построить минимум три варианта нейросетей (моделей)
* для тестовой выборки построить матрицу ошибок(матрица путаницы) - сделать выводы о качестве модели
* реализовать функцию, которая по входным параметрам возвращает предсказанное с помощью нейросети значение

Для регрессии:
* использовать минимум две метрики для оценки качества модели
* построить минимум три варианта нейросетей (моделей)
* для тестовой выборки построить график сравнения предсказанных значений и с реальными (true vs pred)
* реализовать функцию, которая по входным параметрам возвращает предсказанное с помощью нейросети значение